In [1]:
import real_estate_api as rea

from typing import Union, List
from types import ModuleType
from beartype._decor.decormain import beartype
import warnings

import numpy as np
import pandas as pd
import sys
import importlib

from collections import defaultdict
from itertools import chain
import random

from sklearn.base import TransformerMixin, BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import keys as k
keys = k.getKeys()

In [2]:
user_home = rea.get_UserHome(keys['SampleHouse'].strip("\'"))
hoi = rea.get_HousesOfInterest(user_home, n=2000, listed_to_sold_ratio=0.3, verbose=True)
gd = rea.geo_data(hoi['geo'])

Shortfall in listed houses detected, appending 114 of current listing to results.


In [3]:
# Can this be the first step in the pipeline or no?
fg = rea.FeatureGenerator(
    houses = hoi['houses'], 
    gd=gd,
    user_home=user_home
)

In [5]:
from sklearn.preprocessing import StandardScaler

# This will randomly take 10% of the data as a validation set.
test_indicies = random.sample(range(0, len(fg.targets)), int(0.1 * len(fg.targets)))
train_indicies = set(range(0, len(fg.targets))).difference(set(test_indicies))

target_transformer = StandardScaler()
train_targets = target_transformer.fit_transform(np.array(list(map(fg.targets.__getitem__, train_indicies))).reshape(-1,1))
test_targets = target_transformer.transform(np.array(list(map(fg.targets.__getitem__, test_indicies))).reshape(-1,1))

train_features = list(map(fg.features.__getitem__, train_indicies))
test_features = list(map(fg.features.__getitem__, test_indicies))

In [6]:
'''
Days Listed - Linear
Days Updated - Linear
*baths - Normalize
year_built - I want to bucketize these, then keep the dummies.
lot_sqft - Normalize
    Feature generation, multiply lot_sqft normalized and the inverse_distance
        More questions, can you do this?
sqft - Normalize
garage / stories / beds - Normalize
Tags - Do what we did in the nlp homework
'''

from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

from collections import Counter

class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X).drop_duplicates(subset=['Property_ID'])

class DictEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tags_to_keep = None

    def fit(self, X, y=None):
        tag_frequency = Counter(chain(*X))
        self.tags_to_keep = [k for k, v in tag_frequency.items() if v > 1]
        return self
    
    def transform(self, X):
        return pd.Series(map(lambda l : {k : 1 for k in l if l in self.tags_to_keep}, X))

# We need to impute the lat and long

minmax_cols = ['distance_to_home'] # 'Days_listed', 'Days_listed', 
normalize_cols = ['lot_sqft', 'sqft']
bucketize_cols = ['year_built']
lat_long_cols = ['lat', 'long']
dummy_cols = ['baths_full', 'baths_full', 'baths_full', 'baths_1qtr', 'garage', 'stories', 'beds']

preprocess_min_max_cols = Pipeline(
    [
        ('impute', SimpleImputer(missing_values=np.nan, strategy="mean")),
        ('min_max_scale', MinMaxScaler())
    ]
)

preprocess_tags_col = Pipeline(
    [
        ('dict_encode', DictEncoder()),
        ('dict_vectorize', DictVectorizer())
    ]
)

preprocess_lat_long_cols = Pipeline(
    [
        ('impute', SimpleImputer(missing_values=np.nan, strategy="mean")),
        ('bucketize', KBinsDiscretizer(strategy='uniform'))
    ]
)

preprocess_data = ColumnTransformer(
    [
        ('scale', preprocess_min_max_cols, minmax_cols),
        ('normalize', StandardScaler(), normalize_cols),
        ('bucketize', KBinsDiscretizer(strategy='uniform'), bucketize_cols),
        ('lat_long', preprocess_lat_long_cols, lat_long_cols),
        ('dummy', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), dummy_cols),
        #('list', preprocess_tags_col, 'tags')
    ]
)

In [13]:
import os
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.pipeline import FunctionTransformer, FeatureUnion
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.losses import MeanSquaredLogarithmicError

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

class KerasModel(BaseEstimator, RegressorMixin):
    def __init__(self, user_home, load_model_if_available = True, update_model = False):
        self.user_home = user_home
        self.load_model_if_available = load_model_if_available
        self.update_model = update_model

        self.earlystopping = EarlyStopping(patience=5, verbose=1, min_delta=0.1)

        if update_model and not load_model:
            raise Exception('Can not update a model not loaded.')

        self.model_name = f'{user_home.get("city")}_{user_home.get("state_code")}'

    def _keras_model(self, n_cols):
        km = Sequential()
        km.add(Dense(256, input_shape=(n_cols,), activation='relu', kernel_initializer='normal', name='dense_1'))
        km.add(Dense(128, activation='relu', kernel_initializer='normal', name='dense_2'))
        #km.add(Dropout(0.20, name='dropout'))
        km.add(Dense(64, activation='relu', kernel_initializer='normal', name='dense_3'))
        km.add(Dense(1, activation='linear', kernel_initializer='normal', name='output'))
        km.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])
        return KerasRegressor(model=km)

    def _generate_model(self, X, y):
            if not self.update_model: 
                self.model = self._keras_model(n_cols=X.shape[1])
            self.model.fit(X, y, epochs=50, batch_size = 50, callbacks = self.earlystopping)

    def fit(self, X, y=None):
        model_file_path = f'Saved Results/KerasModel/{self.model_name}.pkl'

        if self.load_model_if_available:
            if os.path.isfile(model_file_path):
                with open(model_file_path, 'rb') as f:
                    self.model = pickle.load(f)
                if self.update_model:
                    self._generate_model(X, y)
            else:
                print(f'No model found, generating {self.model_name}.')
                self._generate_model(X, y)
        else:
            self._generate_model(X, y)

        if self.update_model or not self.load_model_if_available:
            with open(model_file_path, 'wb') as f:
                pickle.dump(self.model, f)
        
        return self

    def predict(self, X, tt = target_transformer):
        return tt.inverse_transform(self.model.predict(X))

keras_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('keras_model', KerasModel(user_home))
    ]
)

keras_pipeline.fit(train_features, train_targets)
# I need to figure out hot to pass parameters into this thing.

No model found, generating Seattle_WA.
Epoch 1/50
50/50 [==============================] - 2s 2ms/step - loss: 0.5559 - mean_squared_error: 0.5559 - mean_absolute_error: 0.4921
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2688 - mean_squared_error: 0.2688 - mean_absolute_error: 0.3566
Epoch 3/50
50/50 [==============================] - 0s 3ms/step - loss: 0.2554 - mean_squared_error: 0.2554 - mean_absolute_error: 0.3530
Epoch 4/50
50/50 [==============================] - 0s 3ms/step - loss: 0.2385 - mean_squared_error: 0.2385 - mean_absolute_error: 0.3377
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2420 - mean_squared_error: 0.2420 - mean_absolute_error: 0.3452
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2214 - mean_squared_error: 0.2214 - mean_absolute_error: 0.3325
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1976 - mean_squared_error: 0.1976 - mean_absolute_error: 0.

Pipeline(steps=[('to_data_frame', ToDataFrame()),
                ('preprocess',
                 ColumnTransformer(transformers=[('scale',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('min_max_scale',
                                                                   MinMaxScaler())]),
                                                  ['distance_to_home']),
                                                 ('normalize', StandardScaler(),
                                                  ['lot_sqft', 'sqft']),
                                                 ('bucketize',
                                                  KBinsDiscretizer(strategy='uniform'),
                                                  ['year_built']),
                                                 ('lat_long',
                                                  Pipeline(steps=[('...
                                       'area_type': 'address',
                                       'centroid': {'lat': 47.572613,
                                                    'lon': -122.306358},
                                       'city': 'Seattle', 'country': 'USA',
                                       'full_address': ['3416 19th Ave S '
                                                        'Seattle WA 98144',
                                                        '3416 19th Ave S '
                                                        'Seattle WA'],
                                       'line': '3416 19th Ave S',
                                       'mpr_id': '1897471296',
                                       'postal_code': '98144',
                                       'prop_status': ['recently_sold',
                                                       'off_market'],
                                       'property_id': '1897471296',
                                       'state_code': 'WA',
                                       'validation_code': ['121']}))])

In [15]:
#best_keras.best_estimator_.score(test_features, target_transformer.inverse_transform(test_targets))
preds = keras_pipeline.predict(test_features)
mean_absolute_error(preds, target_transformer.inverse_transform(test_targets))

6/6 [==============================] - 0s 1ms/step


257407.7975